In [1]:
import numpy as np
import pandas as pd
import collections
from scipy import stats
import matplotlib.pyplot as plt 

In [2]:
pd.set_option('precision', 1)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

### Reading and cleaning data 

#### University Towns

In [3]:
Utowns = pd.read_fwf('university_towns.txt',header = None)
Utowns.columns = ['RegionName']
Utowns.head(15)

,RegionName
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


We can see that college towns are mixed with States - we have to clean it up and separate states from uni towns in the first run:

In [4]:
### searching for index and name of states in Utowns
state_idx = []
state_name= []
for i,state in enumerate(Utowns['RegionName']):
    if '[edit]' in state:
        state_idx.append(i)
        state = state[:state.rfind('[')] #cleaning state name by erasing '[edit]' part from string
        state_name.append(state)
        
### adding last index to bins in state_idx        
state_idx.append(len(Utowns))

state_idx[:10],state_name[:10]

([0, 9, 11, 15, 24, 50, 60, 68, 71, 84],
 ['Alabama',
  'Alaska',
  'Arizona',
  'Arkansas',
  'California',
  'Colorado',
  'Connecticut',
  'Delaware',
  'Florida',
  'Georgia'])

Now we have indexes for every state and cleaned name of states. Next step is to construct Data Frame, with the
following format:
    
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
Every region will have corresponding state

In [5]:
### using cut function to create new col with state name
Utowns['state_idx'] = pd.cut(Utowns.index,state_idx,right=False,include_lowest=True,labels = state_name)        
Utowns.rename(columns = {'state_idx':'State'},inplace=True)    
Utowns = Utowns[['State','RegionName']]

Defining cleaning function to get rid of every character after '(', for example:<br>
in: Troy (Troy University)<br>
out: Troy

In [6]:
def clean_fun(char):
    
    if char.rfind('(') != -1:
        char = char[:char.rfind('(')-1]
    return char    

In [7]:
### cleaning RegionName col
Utowns['RegionName'] = Utowns['RegionName'].map(clean_fun)
Utowns.head(3)

,State,RegionName
0,Alabama,Alabama[edit]
1,Alabama,Auburn
2,Alabama,Florence


In [8]:
### choosing regions that are not states (row selection)
Utowns = Utowns[Utowns.index.isin(state_idx[:-1]) == False]
Utowns = Utowns.reset_index(drop=True)

Utowns['RegionName'] = Utowns['RegionName'].astype(object) 
Utowns['State'] =  Utowns['State'].astype(object)

After few manipulations above we have DataFrame with university town and its state:

In [9]:
Utowns.head(7)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa


#### House prices

In [10]:
Housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')

In [11]:
Housing_df.head(3)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,644300,655300,667800,677500,683400,688300,694800,701900,707700,710300
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,621700,626600,630200,632500,633800,636700,642100,647600,653500,658500
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,219500,220000,220300,220400,220800,221500,221700,221400,220900,221000


As we can see we need to convert State abbreviations to its full names to match it with Utowns DataFrame (we will use RegionName and State columns in Utowns and Housing_df as keys to merge both tables later)

In [12]:
### converting states abbreviations to its full names:
### dictionary with mapping from abbreviation to state full name
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 
          'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
          'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 
          'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 
          'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina',
          'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado',
          'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island',
          'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts',
          'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [13]:
Housing_df['State'] = [states[x] for x in Housing_df['State']]

In [14]:
Housing_df['State'].head()

0        New York
1      California
2        Illinois
3    Pennsylvania
4         Arizona
Name: State, dtype: object

State and RegionName will be set as an index <br>
From 4th column on, there are consecutive months with house price indices starting from 04-1996 <br>
We have to transform monthly to quarterly periods, we will use mean function as an aggregating funcition

In [15]:
Housing_df = Housing_df.set_index(['State','RegionName'])
Housing_dfQ = Housing_df.iloc[:,4:].groupby(pd.PeriodIndex(Housing_df.columns[4:], freq='Q'), axis=1).mean()

In [16]:
Housing_dfQ.head(3)

,,1996Q2,1996Q3,1996Q4,1997Q1,1997Q2,1997Q3,1997Q4,1998Q1,1998Q2,1998Q3,...,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,557166.7,569266.7,579900.0,589033.3,601966.7,617833.3,635433.3,666866.7,688833.3,706633.3
California,Los Angeles,154666.7,154200.0,154433.3,156866.7,158533.3,159266.7,162000.0,166966.7,171400.0,175966.7,...,543433.3,554266.7,565700.0,579633.3,588666.7,602133.3,616066.7,629766.7,637533.3,653200.0
Illinois,Chicago,109466.7,109133.3,109600.0,111266.7,112200.0,112966.7,114833.3,116966.7,118433.3,120700.0,...,200566.7,202133.3,203200.0,204966.7,207066.7,211866.7,218466.7,220233.3,221333.3,221100.0


In accordance with task in this assignment, we are interested with periods from 2000Q1 onwards.

In [17]:
Housing_dfQ = Housing_dfQ.loc[:,'2000Q1':]

In [18]:
Housing_dfQ.head()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,557166.7,569266.7,579900.0,589033.3,601966.7,617833.3,635433.3,666866.7,688833.3,706633.3
California,Los Angeles,207066.7,214466.7,220966.7,226166.7,233033.3,239100.0,245066.7,253033.3,261966.7,272700.0,...,543433.3,554266.7,565700.0,579633.3,588666.7,602133.3,616066.7,629766.7,637533.3,653200.0
Illinois,Chicago,138400.0,143633.3,147866.7,152133.3,156933.3,161800.0,166400.0,170433.3,175500.0,177566.7,...,200566.7,202133.3,203200.0,204966.7,207066.7,211866.7,218466.7,220233.3,221333.3,221100.0
Pennsylvania,Philadelphia,53000.0,53633.3,54133.3,54700.0,55333.3,55533.3,56266.7,57533.3,59133.3,60733.3,...,119700.0,121166.7,123066.7,125633.3,130033.3,131800.0,134666.7,136666.7,139433.3,143566.7
Arizona,Phoenix,111833.3,114366.7,116000.0,117400.0,119600.0,121566.7,122700.0,124300.0,126533.3,128366.7,...,180233.3,184433.3,188800.0,193233.3,198166.7,202433.3,207766.7,212333.3,218266.7,223000.0


#### US GDP quarterly

After screening gdplev excel file, we can see, that data that is iteresting for is starts from 8th row and from 5th column (so in Python with 4th index)

In [19]:
gdplev = pd.read_excel('gdplev.xlsx', skiprows = 7)

In [20]:
gdplev.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1929.0,104.6,1056.6,NaN,1947Q1,243.1,1934.5,NaN
1,1930.0,92.2,966.7,NaN,1947Q2,246.3,1932.3,NaN
2,1931.0,77.4,904.8,NaN,1947Q3,250.1,1930.3,NaN


In [21]:
gdplev = gdplev.iloc[:,[4,6]]
gdplev.columns = ['Quarter','GDP']

In [22]:
gdplev.head()

,Quarter,GDP
0,1947Q1,1934.5
1,1947Q2,1932.3
2,1947Q3,1930.3
3,1947Q4,1960.7
4,1948Q1,1989.5


#### Defining recession start, recession end and recession bottom

To test the hipothesis stated in this assignment that University towns have their mean housing prices less effected by recessions we need to properly find recession periods and measure the price_ratio defined: 

(price_ratio=quarter_before_recession/recession_bottom)

In order to measure price_ratio we need to find:  
* recession start date
* recession end date
* recession bottom

In [23]:
def recession_start_end(series):
    ''' returns tuple of two pd.Series, where the first element is recession start Series (with index 
        and quarter of recession start) and second element is recession end Series (with index and
        quarter of recession end)
    '''
    rec_started = False  #flag whether recession has already started
    recession_start=[] 
    recession_end =[]
    
    for q in range(2,len(series)-2):
        
        #check whether recession has started and is not a continuation of ongoing recession:
        if (series.iloc[q,1] < series.iloc[q-1,1] < series.iloc[q-2,1]) and (rec_started) == False: 
            recession_start.append(q)
            rec_started = True
        
        #if recession is ongoing: 'rec_started == True', we check when it has ended:
        elif (series.iloc[q,1] < series.iloc[(q+1),1] < series.iloc[(q+2),1]) and (rec_started) == True:
            recession_end.append(q+2)
            rec_started = False
    
    rec_start_series = pd.Series(series.iloc[recession_start,0])
    rec_end_series = pd.Series(series.iloc[recession_end,0])

    return rec_start_series, rec_end_series

In [24]:
def recession_bottom(series):
    ''' returns pd.Series with indexes from 'series' and quarters of recession bottom
    '''

    start_to_end =  list(zip(rec_start.index,rec_end.index))
    rec_bottom_idx = [series.iloc[x[0]:x[1],1].argmin() for x in start_to_end]
    rec_bottom = pd.Series(gdplev.iloc[rec_bottom_idx,0])
        
    return rec_bottom

In [25]:
rec_start = recession_start_end(gdplev)[0]  
rec_end = recession_start_end(gdplev)[1] 
rec_bottom = recession_bottom(gdplev)

In [26]:
rec_start

2      1947Q3
9      1949Q2
27     1953Q4
44     1958Q1
92     1970Q1
111    1974Q4
134    1980Q3
176    1991Q1
247    2008Q4
Name: Quarter, dtype: object

In [27]:
rec_end

5      1948Q2
13     1950Q2
30     1954Q3
47     1958Q4
97     1971Q2
114    1975Q3
144    1983Q1
179    1991Q4
251    2009Q4
Name: Quarter, dtype: object

In [28]:
rec_bottom

2      1947Q3
9      1949Q2
28     1954Q1
44     1958Q1
92     1970Q1
112    1975Q1
134    1980Q3
176    1991Q1
249    2009Q2
Name: Quarter, dtype: object

Since the beginnig of XX century there was only 1 recession in US, so we are interested in last element in rec_start, rec_end, rec_bottom series:

In [29]:
last_rec =  pd.Period(rec_start.iloc[-1])
last_before_rec = pd.Period(rec_start.iloc[-1])-1
last_bot =  pd.Period(rec_bottom.iloc[-1])
last_end = pd.Period(rec_end.iloc[-1])

In [30]:
pd.options.display.float_format = '{:,.1f}'.format
Housing_dfQ.loc[:,last_rec:last_bot].head()

,,2008Q4,2009Q1,2009Q2
State,RegionName,,,
New York,New York,"487,400.0","476,000.0","465,966.7"
California,Los Angeles,"454,066.7","435,966.7","417,300.0"
Illinois,Chicago,"227,633.3","224,300.0","219,533.3"
Pennsylvania,Philadelphia,"116,866.7","117,133.3","116,433.3"
Arizona,Phoenix,"185,766.7","177,400.0","167,666.7"


Now we need to merge Utowns and Housing_dfQ frames to get index values for uni towns

In [31]:
Utowns_prices = pd.merge(Utowns, Housing_dfQ, how='left',left_on=['State','RegionName'], right_index=True)

In [32]:
Utowns_prices.head()

,State,RegionName,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,...,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4
0,Alabama,Auburn,"157,266.7","159,066.7","157,666.7","157,133.3","156,533.3","162,466.7","168,466.7","171,100.0",...,"213,733.3","215,500.0","217,700.0","221,433.3","223,966.7","226,033.3","232,566.7","237,466.7","239,466.7","241,533.3"
1,Alabama,Florence,"75,566.7","76,233.3","77,333.3","76,633.3","77,800.0","78,700.0","78,700.0","78,900.0",...,"101,633.3","102,533.3","102,600.0","104,066.7","105,200.0","105,866.7","106,433.3","108,633.3","110,166.7","111,233.3"
2,Alabama,Jacksonville,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Alabama,Livingston,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Alabama,Montevallo,"97,000.0","96,800.0","96,533.3","98,333.3","99,466.7","101,333.3","103,200.0","101,866.7",...,"123,200.0","124,033.3","124,866.7","124,600.0","123,000.0","124,000.0","127,133.3","129,500.0","131,833.3","132,433.3"


Then we need to exclude from Housing_dfQ rows that are present in Utowns frame (to get Non-university towns).
In order to achieve it I will add 'ones_housing' and 'ones_utowns' column to Housign_dfQ and Utowns data frames correspondingly and then merge those two datasets and choose only those records that have NaN values in 'ones_Utowns' column (so I will exclude records that were in Utowns and not in Housing_dfQ) 

In [33]:
### adding dummy columns with ones - it will be used to exclude towns from Utowns 
### that are in Housing_dfQ frame

Housing_dfQ.columns = Housing_dfQ.columns.map(str)
Housing_dfQ["ones_housing"] = 1
Utowns['ones_utowns'] = 1

In [34]:
Housing_dfQ.columns

Index(['2000Q1', '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2', '2001Q3',
       '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4', '2003Q1', '2003Q2',
       '2003Q3', '2003Q4', '2004Q1', '2004Q2', '2004Q3', '2004Q4', '2005Q1',
       '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2', '2006Q3', '2006Q4',
       '2007Q1', '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
       '2008Q4', '2009Q1', '2009Q2', '2009Q3', '2009Q4', '2010Q1', '2010Q2',
       '2010Q3', '2010Q4', '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012Q1',
       '2012Q2', '2012Q3', '2012Q4', '2013Q1', '2013Q2', '2013Q3', '2013Q4',
       '2014Q1', '2014Q2', '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3',
       '2015Q4', '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2',
       '2017Q3', '2017Q4', 'ones_housing'],
      dtype='object')

In [35]:
Utowns.head()

,State,RegionName,ones_utowns
0,Alabama,Auburn,1
1,Alabama,Florence,1
2,Alabama,Jacksonville,1
3,Alabama,Livingston,1
4,Alabama,Montevallo,1


In [36]:
NonUtowns_prices = pd.merge(Housing_dfQ, Utowns, how='left', left_index=True,right_on=['State','RegionName'])
NonUtowns_prices = NonUtowns_prices[NonUtowns_prices['ones_utowns'].isnull()]

In [37]:
Uratio = Utowns_prices.loc[:,last_rec-1]/Utowns_prices.loc[:,last_bot]

In [38]:
NonUratio = NonUtowns_prices.loc[:,str(last_rec-1)]/NonUtowns_prices.loc[:,str(last_bot)] 

In [39]:
def testing_diff(Uratio, NonUratio):
    
    t,p = stats.ttest_ind(Uratio,NonUratio,axis=0,nan_policy='omit',equal_var=False)
    significance = False
    if p < 0.01:
        significance = True

    less_effected = 'Non-university towns'
    if np.mean(Uratio) < np.mean(NonUratio):
        less_effected = 'University towns'

    return (significance, p, less_effected)

In [40]:
result = testing_diff(Uratio,NonUratio)

In [41]:
print('Wald t_test is significant: \t\t{0}\n'
       'p-value equals: \t\t\t{1}\n'
       'Less effected by recession are: \t{2}\n'
      .format(result[0],result[1],result[2]))

Wald t_test is significant: 		True
p-value equals: 			0.003819769797750409
Less effected by recession are: 	University towns



#### Conclusion: We need to reject the null hipothesis, that there is no difference between mean Uratio and NonUratio. 

Mean reduced market loss was lower for University towns in USA during last recession started in 2008.  Difference is significant and there is <0.01 probability of making type I error (rejecting null hipothesis that is actually true)